# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.70it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.73it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.31it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.18it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.33it/s]



...

 83%|████████▎ | 19/23 [00:07<00:01,  2.96it/s]

 87%|████████▋ | 20/23 [00:07<00:00,  3.15it/s]

 91%|█████████▏| 21/23 [00:08<00:00,  3.16it/s]

 96%|█████████▌| 22/23 [00:08<00:00,  3.16it/s]

100%|██████████| 23/23 [00:08<00:00,  2.65it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Elena and I am the new Teacher at the International School of Zug and Luzern. I am excited to be here and look forward to getting to know all of my new students and colleagues.
I have a degree in Early Childhood Education from the University of Athens and a Master's degree in Special Education from the University of New Mexico. I have been teaching for over 15 years and have experience working with students of all ages and backgrounds.
My classroom will be a collaborative and engaging environment where students are encouraged to explore, learn, and grow. I believe that every student is unique and talented and I will work with each student to help them reach
Prompt: The president of the United States is
Generated text:  the head of state and head of government of the United States. The president is directly elected by the people through the Electoral College. The president serves a four-year term, and can be re-elected once. The president has t

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 K

ait

lyn

.


...

.

 In

 contrast

,

 multi

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Claire. I'm a 23-year-old American living in the south of Spain. I've been here for about 6 months, and I have to say, it's been a rollercoaster of emotions. I'm not sure if I'm fully embracing the Spanish way of life, or if I'm just pretending to be a local. Either way, I've learned a lot about myself and the culture.
My friends and I often have heated debates about the pros and cons of living in Spain. Some people love it here, while others, like me, are still figuring things out. We've talked about everything from the siesta

Prompt: The capital of France is
Generated text:  Paris. While the majority of visitors to Paris come for the city’s culture, art, and history, there are numerous things to do and see in the city’s surroundings. The Louvre Museum, the Arc de Triomphe, and the Eiffel Tower are some of the most well-known Parisian landmarks. The city is famous for its cuisine, including croissants, cheese, and wine.
The Eiffel Tower, on

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Kirst

y

 and

 I

...

 specific

 environment

,

 such

 as

In [6]:
llm.shutdown()